## 课时28 Cycle_GAN代码实现(苹果转橘子)

In [43]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
tf.__version__

'1.13.1'

## 1. 定义Generator模块(CycleGAN中的Generator也是U-Net网络)

In [44]:
def generator(inputs_real, is_train=True, alpha=0.01, name="generator"):
    # 由于Cycle_GAN中有两个Generator网络(G_AB和G_BA)，为了代码复用，所以定义了name这个参数
    # 通过name不同的赋值来区分不同的生成器Generator，以达到代码复用的目的
    # 256*256*3
    with tf.variable_scope(name, reuse=(not is_train)):
        # 128*128*64
        conv1 = tf.layers.conv2d(inputs_real, 64, (3,3), padding='same')
        conv1 = tf.nn.relu(conv1)
        conv1 = tf.layers.max_pooling2d(conv1, (2,2), (2,2), padding='same')
        # 64*64*128
        conv2 = tf.layers.conv2d(conv1, 128, (3,3), padding='same')
        conv2 = tf.nn.relu(conv2)
        conv2 = tf.layers.max_pooling2d(conv2, (2,2), (2,2), padding='same')
        # 32*32*256
        conv3 = tf.layers.conv2d(conv2, 256, (3,3), padding='same')
        conv3 = tf.nn.relu(conv3)
        conv3 = tf.layers.max_pooling2d(conv3, (2,2), (2,2), padding='same')
        # 16*16*512
        conv4 = tf.layers.conv2d(conv3, 512, (3,3), padding='same')
        conv4 = tf.nn.relu(conv4)
        conv4 = tf.layers.max_pooling2d(conv4, (2,2), (2,2), padding='same')
        # 8*8*512
        conv5 = tf.layers.conv2d(conv4, 512, (3,3), padding='same')
        conv5 = tf.nn.relu(conv5)
        conv5 = tf.layers.max_pooling2d(conv5, (2,2), (2,2), padding='same')
        # 4*4*512
        conv6 = tf.layers.conv2d(conv5, 512, (3,3), padding='same')
        conv6 = tf.nn.relu(conv6)
        conv6 = tf.layers.max_pooling2d(conv6, (2,2), (2,2), padding='same')
        # 2*2*512
        conv7 = tf.layers.conv2d(conv6, 512, (3,3), padding='same')
        conv7 = tf.nn.relu(conv7)
        conv7 = tf.layers.max_pooling2d(conv7, (2,2), (2,2), padding='same')
        # 1*1*512
        conv8 = tf.layers.conv2d(conv7, 512, (3,3), padding='same')
        conv8 = tf.nn.relu(conv8)
        conv8 = tf.layers.max_pooling2d(conv8, (2,2), (2,2), padding='same')
        
        
        # 2*2*512
        conv9 = tf.layers.conv2d_transpose(conv8, 512, 3, strides=2, padding='same')
        conv9 = tf.layers.batch_normalization(conv9, training=is_train)
        conv9 = tf.nn.relu(conv9)
        conv9 = tf.nn.dropout(conv9, keep_prob=0.5)
        # 4*4*512
        conv10 = tf.concat([conv9,conv7], 3)
        conv10 = tf.layers.conv2d_transpose(conv10, 512, 3, strides=2, padding='same')
        conv10 = tf.layers.batch_normalization(conv10, training=is_train)
        conv10 = tf.nn.relu(conv10)
        conv10 = tf.nn.dropout(conv10, keep_prob=0.5)
        # 8*8*512
        conv11 = tf.concat([conv10,conv6], 3)
        conv11 = tf.layers.conv2d_transpose(conv11, 512, 3, strides=2, padding='same')
        conv11 = tf.layers.batch_normalization(conv11, training=is_train)
        conv11 = tf.nn.relu(conv11)
        conv11 = tf.nn.dropout(conv11, keep_prob=0.5)
        # 16*16*512
        conv12 = tf.concat([conv11,conv5], 3)
        conv12 = tf.layers.conv2d_transpose(conv12, 512, 3, strides=2, padding='same')
        conv12 = tf.layers.batch_normalization(conv12, training=is_train)
        conv12 = tf.nn.relu(conv12)
        # 32*32*256
        conv13 = tf.concat([conv12,conv4], 3)
        conv13 = tf.layers.conv2d_transpose(conv13, 256, 3, strides=2, padding='same')
        conv13 = tf.layers.batch_normalization(conv13, training=is_train)
        conv13 = tf.nn.relu(conv13)
        # 64*64*128
        conv14 = tf.concat([conv13,conv3], 3)
        conv14 = tf.layers.conv2d_transpose(conv14, 128, 3, strides=2, padding='same')
        conv14 = tf.layers.batch_normalization(conv14, training=is_train)
        conv14 = tf.nn.relu(conv14)
        # 128*128*64
        conv15 = tf.concat([conv14,conv2], 3)
        conv15 = tf.layers.conv2d_transpose(conv15, 64, 3, strides=2, padding='same')
        conv15 = tf.layers.batch_normalization(conv15, training=is_train)
        conv15 = tf.nn.relu(conv15)
        # 256*256*3
        conv16 = tf.concat([conv15,conv1], 3)
        conv16 = tf.layers.conv2d_transpose(conv16, 3, 3, strides=2, padding='same')
        # conv16 = tf.layers.batch_normalization(conv16, training=is_train)
    
        # 图片归一化(将图像的值转换到[-1, 1]区间)
        outputs = tf.nn.tanh(conv16)
        
        return outputs

## 2. 定义Discriminator模块(CycleGAN中的Discriminator也有两个网络)

In [45]:
def discriminator(inputs_image, reuse=False, alpha=0.01, name="discriminator"):
    # 由于Discriminator也是两个网络，所以为了能够复用代码，也是通过name来进行判别器区分的
    with tf.variable_scope(name, reuse=reuse):
        layer1 = tf.layers.conv2d(inputs_image, 64, 3, strides=2, padding='same')
        layer1 = tf.layers.batch_normalization(layer1, training=True)
        layer1 = tf.maximum(alpha * layer1, layer1)
        
        layer2 = tf.layers.conv2d(layer1, 64, 3, strides=2, padding='same')
        layer2 = tf.layers.batch_normalization(layer2, training=True)
        layer2 = tf.maximum(alpha * layer2, layer2)
        
        layer3 = tf.layers.conv2d(layer2, 128, 3, strides=2, padding='same')
        layer3 = tf.layers.batch_normalization(layer3, training=True)
        layer3 = tf.maximum(alpha * layer3, layer3)
        
        layer4 = tf.layers.conv2d(layer3, 128, 3, strides=2, padding='same')
        layer4 = tf.layers.batch_normalization(layer4, training=True)
        layer4 = tf.maximum(alpha * layer4, layer4)
        
        layer5 = tf.layers.conv2d(layer4, 256, 3, strides=2, padding='same')
        layer5 = tf.layers.batch_normalization(layer5, training=True)
        layer5 = tf.maximum(alpha * layer5, layer5)
        
        layer6 = tf.layers.conv2d(layer5, 256, 3, strides=2, padding='same')
        layer6 = tf.layers.batch_normalization(layer6, training=True)
        layer6 = tf.maximum(alpha * layer6, layer6)
        
        layer7 = tf.layers.conv2d(layer6, 512, 3, strides=2, padding='same')
        layer7 = tf.layers.batch_normalization(layer7, training=True)
        layer7 = tf.maximum(alpha * layer7, layer7)
        
        flatten = tf.reshape(layer7, (-1, 2*2*512))
        logits = tf.layers.dense(flatten, 1)
        outputs = tf.sigmoid(logits)
        
        return  logits, outputs

## 3. 定义Loss损失函数

In [58]:
# Cycle_GAN中有四个loss
def get_loss(inputs_images, inputs_cartoons, smooth=0.1):
    """
    inputs_images: 实际指的就是inputs_apple
    inputs_cartoons: 实际指的就是inputs_oringe
    """
    # 输入inputs_images(真实的苹果图片) ===> 生成fake_cartoons(生成的橘子图片)
    fake_cartoons = generator(inputs_images, name="generatorI2C")
    # 输入fake_cartoons(生成的橘子图片) ===> 生成fake_images_(再生成的苹果图片)
    fake_images_ = generator(fake_cartoons, name="generatorC2I")
    # ====================================================================
    fake_images = generator(inputs_cartoons, False, name="generatorC2I")
    fake_cartoons_ = generator(fake_images, False, name="generatorI2C")
    
    # 这个判别器判断输入的图片是真实的橘子还是生成的橘子
    discriminator_cartoon_fake, cartoon_fake_logits = discriminator(fake_cartoons, 
                                                                    reuse=False, 
                                                                    name="discriminator_cartoon")
    discriminator_cartoon_real, cartoon_real_logits = discriminator(inputs_cartoons, 
                                                                    reuse=True, 
                                                                    name="discriminator_cartoon")
    # 这个判别器判断输入的图片是否是真实的橘子或者苹果
    discriminator_image_fake, image_fake_logits = discriminator(fake_images, 
                                                                reuse=False, 
                                                                name="discriminator_image")
    discriminator_image_real, image_real_logits = discriminator(inputs_images, 
                                                                reuse=True, 
                                                                name="discriminator_image")
    
    
    
    
    
    # 定义generator的损失loss
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=cartoon_fake_logits, 
                                                                    labels=tf.ones_like(discriminator_cartoon_fake)*(1-smooth))) \
            + tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=image_fake_logits,
                                                                    labels=tf.ones_like(discriminator_image_fake)*(1-smooth))) \
            + tf.reduce_mean(tf.abs(inputs_images - fake_images_))\
            + tf.reduce_mean(tf.abs(inputs_cartoons - fake_cartoons_))
            
    
    
    
    d_cartoon_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=cartoon_real_logits, 
                                                                                 labels=tf.ones_like(discriminator_cartoon_real)*(1-smooth)))
    d_cartoon_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=cartoon_fake_logits, 
                                                                                 labels=tf.zeros_like(discriminator_cartoon_fake)))
    # =============================================================================================
    d_image_real_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=image_real_logits, 
                                                                               labels=tf.ones_like(discriminator_image_real)*(1-smooth)))
    d_image_fake_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=image_fake_logits, 
                                                                               labels=tf.zeros_like(discriminator_image_fake)))
    
    d_loss = d_cartoon_real_loss + d_image_real_loss + d_cartoon_fake_loss + d_image_fake_loss

    
    return g_loss, d_loss

## 4. 定义优化器

In [47]:
def get_optimizer(g_loss, d_loss, g_loss_tran, learning_rate=0.0001):
    
    train_vars = tf.trainable_variables()
    
    g_vars = [var for var in train_vars if var.name.startswith("generator")]
    d_vars = [var for var in train_vars if var.name.startswith("discriminator")]
    
    
    # Optimizer
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        g_opt = tf.train.AdamOptimizer(learning_rate*5).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
    
    return g_opt, d_opt

## 5. 定义辅助函数

In [48]:
def plot_images(samples):
    samples = (samples + 1) / 2
    fig, axes = plt.subplots(nrows=1, ncols=10, sharex=True, sharey=True, figsize=(10,1))
    for img, ax in zip(samples, axes):
        ax.imshow(img)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    fig.tight_layout(pad=0)

In [49]:
def show_generator_output(sess, samp_images):
    samples = sess.run(generator(samp_images, False , name="generatorI2C"))
    samples = sess.run(tf.reshape(samples, [-1, 256, 256, 3]))
    return samples

## 6. 定义训练函数

In [31]:
# 定义参数
beta1 = 0.4
learning_rate = 0.0001

In [60]:
def train():
    
    # 存储loss
    losses = []
    steps = 1000
    
    apple_list = glob.glob('../日月光华-GAN大型数据集/1.0部分代码和数据集/数据集/apple2orange/trainA/*.jpg')
    orange_list = glob.glob('../日月光华-GAN大型数据集/1.0部分代码和数据集/数据集/apple2orange/trainB/*.jpg')
    image_que = tf.train.slice_input_producer([apple_list, orange_list], shuffle=True)
    
    image_ = tf.read_file(image_que[0])
    image = tf.image.decode_jpeg(image_, channels=3)
    image = tf.image.resize_image_with_crop_or_pad(image, 256, 256)
    new_img = tf.image.convert_image_dtype(image, tf.float32)
    new_img = new_img*2 -1
    
    cartoon_ = tf.read_file(image_que[1])
    cartoon = tf.image.decode_jpeg(cartoon_, channels=3)
    cartoon = tf.image.resize_image_with_crop_or_pad(cartoon, 256, 256)
    # tf.image.convert_image_dtype除了将数据转换为tf.float32之外，还会将数据转换到[0, 1]之间
    new_cartoon = tf.image.convert_image_dtype(cartoon, tf.float32)
    # 为了后续能够跟生成数据进行对比，需要将数据转换到[-1, 1]之间
    new_cartoon = new_cartoon*2 -1
    
    batch_size = 10
    capacity = 3 + 2 * batch_size
          
    image_batch, cartoon_batch = tf.train.batch([new_img, new_cartoon], batch_size=batch_size, capacity=capacity)
    
    g_loss, d_loss = get_loss(image_batch, cartoon_batch)
    g_train_opt, d_train_opt = get_optimizer(g_loss, d_loss, beta1, learning_rate)
    
    
    saver = tf.train.Saver()
    # model_file=tf.train.latest_checkpoint('../input/cycel-apple-to-orange')
    with tf.Session() as sess:
        # saver.restore(sess, model_file)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        sess.run(tf.global_variables_initializer())
        
        
        # 迭代epoch
        
        for e in range(steps):
                # run optimizer
            _ = sess.run(d_train_opt)
            _ = sess.run(g_train_opt)
                
            if e % 100 == 0:
                # saver.save(sess,'./less96', global_step = e)
                train_loss_d = d_loss.eval()
                train_loss_g = g_loss.eval()
                losses.append((train_loss_d, train_loss_g))
                    # 显示图片
                samples = show_generator_output(sess, image_batch)
                plot_images(samples)
                print("Epoch {}/{}....".format(e+1, steps), 
                      "Discriminator Loss: {:.4f}....".format(train_loss_d),
                      "Generator Loss: {:.4f}....". format(train_loss_g))
        saver.save(sess,'../tf_saver_files/class_11_of_cycleGAN/generator.ckpt',global_step = steps)
        coord.request_stop()
        coord.join(threads)                  

In [59]:
with tf.Graph().as_default():
    train()

KeyboardInterrupt: 